# Лабораторная работа 1. Извлечение признаков из текстовых данных


## Задание
- Реализовать алгоритм получения контекстных эмбеддингов (векторных представлений) слов.

- Реализовать один из алгоритмов CBoW или SkipGram для получения собственных моделей Word2Vec для выбранных данных.

- Визуализировать векторные представления в малой размерности.



## Ход работы
1. Загрузить коллекцию текстовых документов: просьбы жильцов. Можете использовать не весь датасет (подмножество строк таблицы). В этой ЛР вас интересует столбец с текстом обращений жильцов. Можете загрузить любой датасет для классификации текстовых данных.

2. Рассмотреть каждый текстовый документ отдельно. Выполнить предварительную обработку текстовых данных (как в ЛР 8 по ML). Токенизация, лемматизация (стемминг), удаление стоп-слов.

3. Составить матрицу контекстных эмбеддингов слов.

4. С помощью PCA уменьшить размер контекстных эмбеддингов (размер задать самостоятельно).

5. С помощью PyTorch реализовать один из алгоритмов Word2Vec: CBoW или SkipGram. Получить векторные представления текстов с помощью собственной реализации одного из этих алгоритмов.

6. Сохранить полученные различными способами векторные представления в файлы tsv. Визуализировать их с помощью сервиса Projector от авторов TensorFlow.

In [3]:
import pandas as pd
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from pymorphy3 import MorphAnalyzer
import string
from gensim.models import Word2Vec
import numpy as np
from sklearn.decomposition import PCA

### 1. Загрузить коллекцию текстовых документов: просьбы жильцов. Можете использовать не весь датасет (подмножество строк таблицы). В этой ЛР вас интересует столбец с текстом обращений жильцов. Можете загрузить любой датасет для классификации текстовых данных.

In [4]:
data = pd.read_csv("../data/Petitions.csv")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59889 entries, 0 to 59888
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    59889 non-null  int64 
 1   public_petition_text  59889 non-null  object
 2   reason_category       59889 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [6]:
data.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [7]:
data = data.head(10000)

In [8]:
data.drop(["id", "reason_category"], axis=1, inplace=True)

### 2. Рассмотреть каждый текстовый документ отдельно. Выполнить предварительную обработку текстовых данных (как в ЛР 8 по ML). Токенизация, лемматизация (стемминг), удаление стоп-слов.

Удаление всех символов, которые не являются цифрами, буквами и пробелами, и токенизация

In [9]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('punkt_tab')

In [10]:
morph = MorphAnalyzer()
stopwords = stopwords.words("russian")


def preProccesingData(text):
    only_letters = re.sub("[^\s^\w]+", "", text)
    tokenized = word_tokenize(only_letters)
    normalized = [morph.normal_forms(word)[0] for word in tokenized]
    no_stop_words = [
        word
        for word in normalized
        if ((word not in string.punctuation) or (word not in stopwords))
    ]
    return no_stop_words

In [11]:
data['public_petition_text'] = data['public_petition_text'].apply(preProccesingData)
data.head()

,public_petition_text
0,"[снег, на, дорога]"
1,"[очистить, кабельный, киоск, от, реклама]"
2,"[просить, убрать, всё, дерево, и, кустарник, к..."
3,"[неудовлетворительный, состояние, парадный, на..."
4,[граффити]


### 3. Составить матрицу контекстных эмбеддингов слов.

In [12]:
model = Word2Vec(sentences=data['public_petition_text'], min_count=1, vector_size=64, epochs=50)

In [13]:
embedding_matrix = []
words = list(model.wv.index_to_key)

for word in words:
    embedding_matrix.append(model.wv[word])

embedding_matrix = np.array(embedding_matrix)


In [17]:
embedding_matrix

array([[-1.4277917 ,  1.6945407 , -0.48240417, ...,  1.3330537 ,
        -0.86998785, -1.7137235 ],
       [ 0.40314075,  1.6296992 , -2.2135773 , ...,  3.2638073 ,
        -1.7394531 ,  0.7052896 ],
       [ 0.03249288,  1.4302204 , -0.2724718 , ..., -0.44682106,
        -0.99575657, -1.3006419 ],
       ...,
       [ 0.11265665,  0.12962668,  0.05227232, ..., -0.01229966,
        -0.09140461,  0.11374051],
       [ 0.14873748,  0.0490455 ,  0.02499888, ..., -0.03449244,
        -0.05412512,  0.02368988],
       [ 0.02410963, -0.08244315, -0.04862258, ..., -0.11299713,
        -0.26015386, -0.22183876]], dtype=float32)

### 4. С помощью PCA уменьшить размер контекстных эмбеддингов (размер задать самостоятельно).

In [20]:
pca = PCA(32)
pca_data = pca.fit_transform(embedding_matrix)

In [22]:
pca_data.shape

(10245, 32)

### 5. С помощью PyTorch реализовать один из алгоритмов Word2Vec: CBoW или SkipGram. Получить векторные представления текстов с помощью собственной реализации одного из этих алгоритмов.